# Choosing window features using LASSO

In this notebook, we will create a large number of window features using a pipeline and then use LASSO as a feature selection method to reduce the number of features we use.


## Data set synopsis


We will work with the hourly electricity demand dataset. It is the electricity demand for the state of Victora in Australia from 2002 to the start of 2015. 

For instructions on how to download, prepare, and store the dataset, refer to notebook number 4, in the folder "01-Create-Datasets" from this repo.


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_context("talk")

# Load data

In [2]:
data = pd.read_csv(
    "../Datasets/victoria_electricity_demand.csv",
    usecols=["demand", "temperature", "date_time"],
    index_col=["date_time"],
    parse_dates=["date_time"],
)

In [3]:
# For this demo we will use a subset of the data
data = data.loc["2010":]

In [4]:
data.head()

,demand,temperature
date_time,,
2010-01-01 00:00:00,8314.448682,21.525
2010-01-01 01:00:00,8267.187296,22.400
2010-01-01 02:00:00,7394.528444,22.150
2010-01-01 03:00:00,6952.047520,21.800
2010-01-01 04:00:00,6867.199634,20.250


# Create lag and window features using a pipeline

In [5]:
from feature_engine.timeseries.forecasting import LagFeatures, WindowFeatures, ExpandingWindowFeatures
from feature_engine.imputation import DropMissingData
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [6]:
df = data.copy()

In [7]:
# Lag features
lag_transformer = LagFeatures(variables=["demand", "temperature"],
                              periods=[1, 2, 3, 24, 24 * 7])

In [8]:
# Window features
window_transformer = WindowFeatures(
    variables=["demand", "temperature"],
    functions=["mean", "std", "kurt", "skew"],
    window=[24, 24 * 7, 24 * 7 * 4, 24 * 7 * 4 * 12],
    periods=1,
)

In [9]:
# Expanding features
expanding_window_transformer = ExpandingWindowFeatures(
    variables=["demand"], 
    functions=["mean", "std", "kurt", "skew"]
)

In [10]:
# Drop missing data introduced by window and lag features
imputer = DropMissingData()

In [11]:
pipe = Pipeline(
    [
        ("lag", lag_transformer),
        ("rolling", window_transformer),
        ("expanding", expanding_window_transformer),
        ("drop_missing", imputer)
    ]
)

df = pipe.fit_transform(df)
df

,demand,temperature,demand_lag_1,temperature_lag_1,demand_lag_2,temperature_lag_2,demand_lag_3,temperature_lag_3,demand_lag_24,temperature_lag_24,...,demand_window_8064_kurt,demand_window_8064_skew,temperature_window_8064_mean,temperature_window_8064_std,temperature_window_8064_kurt,temperature_window_8064_skew,demand_expanding_mean,demand_expanding_std,demand_expanding_kurt,demand_expanding_skew
date_time,,,,,,,,,,,,,,,,,,,,,
2010-12-03 00:00:00,7650.165828,17.825,8311.641438,18.325,8194.758870,18.650,8810.225934,19.000,7594.965872,18.500,...,-0.103898,0.422491,15.744401,5.742699,0.659439,0.778176,9842.090580,1804.188369,-0.103898,0.422491
2010-12-03 01:00:00,7927.140368,17.850,7650.165828,17.825,8311.641438,18.325,8194.758870,18.650,7914.538048,18.375,...,-0.104160,0.422422,15.743942,5.742385,0.660363,0.778423,9841.818798,1804.241597,-0.104074,0.422631
2010-12-03 02:00:00,7327.146056,17.675,7927.140368,17.850,7650.165828,17.825,8311.641438,18.325,7321.428112,17.875,...,-0.104297,0.422397,15.743378,5.741955,0.661546,0.778705,9841.581422,1804.255693,-0.104144,0.422815
2010-12-03 03:00:00,7088.725786,17.625,7327.146056,17.675,7927.140368,17.850,7650.165828,17.825,7045.315052,17.425,...,-0.104314,0.422377,15.742823,5.741552,0.662685,0.778992,9841.269728,1804.361038,-0.104420,0.422872
2010-12-03 04:00:00,7458.314830,17.625,7088.725786,17.625,7327.146056,17.675,7927.140368,17.850,7396.896962,17.275,...,-0.104304,0.422436,15.742305,5.741194,0.663728,0.779267,9840.928560,1804.509421,-0.104740,0.422842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-02-28 19:00:00,9596.777060,28.350,9979.909902,30.850,10258.585392,31.550,10019.921572,31.250,9980.108798,19.700,...,-0.398990,0.291512,15.987450,5.252119,1.053306,0.793359,9463.653128,1752.041445,0.252346,0.499886
2015-02-28 20:00:00,8883.230296,22.200,9596.777060,28.350,9979.909902,30.850,10258.585392,31.550,9411.874558,18.750,...,-0.398995,0.291267,15.988492,5.253737,1.051447,0.793594,9463.656071,1752.022191,0.252414,0.499886
2015-02-28 21:00:00,8320.260550,18.900,8883.230296,22.200,9596.777060,28.350,9979.909902,30.850,8653.510960,18.300,...,-0.398790,0.291143,15.988814,5.254039,1.050537,0.793438,9463.643240,1752.004951,0.252485,0.499911


# Use LASSO to select features

Let's create the target and features. 

In [12]:
# Create target variable
y = df["demand"]

# Drop demand and temperature as features, we do not know them at predict time.
X = df.drop(columns=["demand", "temperature"])

We will apply standard scaling because we are using LASSO.


In [13]:
X_ = StandardScaler().fit_transform(X)
X = pd.DataFrame(data=X_, columns=X.columns)

Note: this is a quick and dirty method, we are leaking data inside the training set by doing this. This is because we are using the mean and standard deviation of the whole data set to do the scaling, however, the values early in time in the training data ought to be blind to future values.

Nevertheless, doing this does not leak any data to any test data set we create and it's quick!

In [14]:
from sklearn.linear_model import Lasso

In [15]:
model = Lasso(alpha=1, random_state=0)
model.fit(X, y)

Lasso(alpha=1, random_state=0)

In [16]:
feature_importances = pd.Series(index=X.columns, data=model.coef_)

In [17]:
feature_importances.abs().sort_values(ascending=False)

demand_lag_1                    2190.963612
demand_lag_2                     877.540173
demand_lag_168                   213.699556
temperature_lag_1                191.079031
demand_window_24_mean            147.367443
temperature_lag_3                103.022075
demand_lag_24                    102.347762
demand_window_24_skew             63.159112
temperature_lag_168               55.488970
demand_window_168_mean            41.249078
demand_lag_3                      35.840390
temperature_window_24_mean        32.765684
demand_window_672_mean            22.018856
temperature_window_24_std         17.669290
temperature_window_24_skew        13.667630
temperature_lag_24                12.553539
demand_expanding_kurt              8.994635
demand_expanding_mean              6.328392
demand_window_672_kurt             5.773869
temperature_window_24_kurt         5.608783
temperature_window_168_skew        5.011450
temperature_window_8064_mean       4.178718
temperature_window_8064_std     

We can see that the lag features are most important but some window features are also selected!